# Inspect data file contents and evaluate their quality

In [7]:
import os
import pandas as pd
from fastparquet import ParquetFile

pd.set_option('display.max_columns', None)

In [8]:
def load_jsonl_gz_data(path: str):
    """
    Load gzip-compressed jsonl data.
    :param str path: Path to file
    :return: DataFrame object of the data
    """

    return pd.read_json(path, lines=True, compression='gzip')


def load_parquet_data(path: str):
    """
    Load parquet data
    :param str path: Path to file
    :return DataFrame object of the data: 
    """

    return ParquetFile(path).to_pandas()

## Load data files to DataFrames for closer inspection

In [62]:
df_first_names = load_jsonl_gz_data('airflow/dags/data/most-popular-first-names-by-municipality.jsonl.gz')
df_municipalities = load_parquet_data('airflow/dags/data/municipality-listing-2020-10-21.parquet')

In [63]:
df_municipalities.head()

,MUNICIPALITY_NUMBER,MUNICIPALITY_NAME_FI,MUNICIPALITY_NAME_SE,MUNICIPALITY_TYPE,PRIMARY_LANGUAGE,ELECTORAL_DISTRICT_NUMBER,ELECTORAL_DISTRICT_NAME_FI,ELECTORAL_DISTRICT_NAME_SE,REGION_NUMBER,REGION_NAME_FI,REGION_NAME_SE
0,5,Alajärvi,Alajärvi,Kaupunki,suomi,10,Vaasan vaalipiiri,Vasa valkrets,13,Etelä-Pohjanmaa,Södra Österbotten
1,9,Alavieska,Alavieska,Kunta,suomi,12,Oulun vaalipiiri,Uleåborgs valkrets,16,Pohjois-Pohjanmaa,Norra Österbotten
2,10,Alavus,Alavo,Kaupunki,suomi,10,Vaasan vaalipiiri,Vasa valkrets,13,Etelä-Pohjanmaa,Södra Österbotten
3,16,Asikkala,Asikkala,Kunta,suomi,6,Hämeen vaalipiiri,Tavastlands valkrets,6,Päijät-Häme,Päijänne-Tavastland
4,18,Askola,Askola,Kunta,suomi,2,Uudenmaan vaalipiiri,Nylands valkrets,1,Uusimaa,Nyland


In [66]:
df_municipalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   MUNICIPALITY_NUMBER         310 non-null    int64 
 1   MUNICIPALITY_NAME_FI        310 non-null    object
 2   MUNICIPALITY_NAME_SE        310 non-null    object
 3   MUNICIPALITY_TYPE           310 non-null    object
 4   PRIMARY_LANGUAGE            310 non-null    object
 5   ELECTORAL_DISTRICT_NUMBER   310 non-null    int64 
 6   ELECTORAL_DISTRICT_NAME_FI  310 non-null    object
 7   ELECTORAL_DISTRICT_NAME_SE  310 non-null    object
 8   REGION_NUMBER               310 non-null    int64 
 9   REGION_NAME_FI              310 non-null    object
 10  REGION_NAME_SE              310 non-null    object
dtypes: int64(3), object(8)
memory usage: 26.8+ KB


In [11]:
df_first_names.head()

,YEAR_OF_BIRTH,BIRTH_MUNICIPALITY_NUMBER,BIRTH_MUNICIPALITY_NAME,GENDER,FIRST_NAME,AMOUNT
0,1969.0,49.0,Espoo,1.0,Mika,39.0
1,1969.0,61.0,Forssa,1.0,Marko,11.0
2,1969.0,91.0,Helsinki,1.0,Mika,256.0
3,1969.0,92.0,Vantaa,1.0,Mika,47.0
4,1969.0,106.0,Hyvinkää,1.0,Mika,16.0


In [ ]:
"""
# Type conversion not possible yet, because int field doesn't allow nulls 

df_first_names_dtypes={
  'YEAR_OF_BIRTH': int,
  'BIRTH_MUNICIPALITY_NUMBER': int,
  'BIRTH_MUNICIPALITY_NAME': str,
  'GENDER': bool,
  'FIRST_NAME': str,
  'AMOUNT': int
}
df_first_names.astype(df_first_names_dtypes)
"""

### Check for null values

In [35]:
df_municipalities.isnull().sum()

MUNICIPALITY_NUMBER           0
MUNICIPALITY_NAME_FI          0
MUNICIPALITY_NAME_SE          0
MUNICIPALITY_TYPE             0
PRIMARY_LANGUAGE              0
ELECTORAL_DISTRICT_NUMBER     0
ELECTORAL_DISTRICT_NAME_FI    0
ELECTORAL_DISTRICT_NAME_SE    0
REGION_NUMBER                 0
REGION_NAME_FI                0
REGION_NAME_SE                0
dtype: int64

In [36]:
df_first_names.isnull().sum()

YEAR_OF_BIRTH                 11
BIRTH_MUNICIPALITY_NUMBER    246
BIRTH_MUNICIPALITY_NAME       11
GENDER                        11
FIRST_NAME                    11
AMOUNT                        11
dtype: int64

Nulls found in file:  `most-popular-first-names-by-municipality.jsonl.gz` <br> 
<br> 
First, let's clean unusable data. Since the above mentioned data set should contain data about given first names, <br> 
we should exclude rows having null values the field: `FIRST_NAME`  


In [18]:
df_first_names_cleaned = df_first_names[df_first_names['FIRST_NAME'].notnull()]

In [38]:
df_first_names_cleaned.isnull().sum()

YEAR_OF_BIRTH                  0
BIRTH_MUNICIPALITY_NUMBER    235
BIRTH_MUNICIPALITY_NAME        0
GENDER                         0
FIRST_NAME                     0
AMOUNT                         0
dtype: int64

Now we will treat the column: `BIRTH_MUNICIPALITY_NUMBER`

In [113]:
filter_nulls = df_first_names_cleaned['BIRTH_MUNICIPALITY_NUMBER'].isnull()
df_first_names_cleaned.loc[filter_nulls, 'BIRTH_MUNICIPALITY_NAME'].value_counts()

Kuopio    119
Oulu      116
Name: BIRTH_MUNICIPALITY_NAME, dtype: int64

Nulls only occur on two cities, which we find also in the data file: `municipality-listing-2020-10-21.parquet`

In [114]:
# Check for missing municipalities' municipality numbers in file 
df_municipalities.loc[df_municipalities['MUNICIPALITY_NAME_FI'].isin(['Oulu', 'Kuopio'])]

,MUNICIPALITY_NUMBER,MUNICIPALITY_NAME_FI,MUNICIPALITY_NAME_SE,MUNICIPALITY_TYPE,PRIMARY_LANGUAGE,ELECTORAL_DISTRICT_NUMBER,ELECTORAL_DISTRICT_NAME_FI,ELECTORAL_DISTRICT_NAME_SE,REGION_NUMBER,REGION_NAME_FI,REGION_NAME_SE
113,297,Kuopio,Kuopio,Kaupunki,suomi,9,Savo-Karjalan vaalipiiri,Savolax-Karelens valkrets,10,Pohjois-Savo,Norra Savolax
183,564,Oulu,Uleåborg,Kaupunki,suomi,12,Oulun vaalipiiri,Uleåborgs valkrets,16,Pohjois-Pohjanmaa,Norra Österbotten


In [122]:
# Create a mapping between municipality names and numbers
df_municipality_mapping = df_first_names_cleaned[['BIRTH_MUNICIPALITY_NUMBER', 'BIRTH_MUNICIPALITY_NAME']].drop_duplicates().sort_values('BIRTH_MUNICIPALITY_NUMBER')

# Try joining data sets by the municipality number
df_merged = df_municipalities.merge(df_municipality_mapping, how='outer', left_on=['MUNICIPALITY_NUMBER', 'MUNICIPALITY_NAME_FI'], right_on=['BIRTH_MUNICIPALITY_NUMBER', 'BIRTH_MUNICIPALITY_NAME'], indicator=True)
df_merged['_merge'].value_counts()

left_only     240
both           70
right_only      5
Name: _merge, dtype: int64

Only five municipalities in the dataset: `most-popular-first-names-by-municipality.jsonl.gz` <br>
are not joinable to dataset `municipality-listing-2020-10-21.parquet` by municipality numbers <br>
<br>
Let's take a closer look into unjoinable municipalities

In [163]:
df_unjoinable_municipalities = df_merged.loc[df_merged['_merge'] == 'right_only', ['BIRTH_MUNICIPALITY_NUMBER', 'BIRTH_MUNICIPALITY_NAME', '_merge']]
list_unjoinable_municipalities = df_unjoinable_municipalities['BIRTH_MUNICIPALITY_NAME'].values
df_unjoinable_municipalities

,BIRTH_MUNICIPALITY_NUMBER,BIRTH_MUNICIPALITY_NAME,_merge
310,88.0,Heinola,right_only
311,427.0,Lohja,right_only
312,612.0,Porvoo,right_only
313,NaN,Kuopio,right_only
314,NaN,Oulu,right_only


In [164]:
list_unjoinable_municipalities

array(['Heinola', 'Lohja', 'Porvoo', 'Kuopio', 'Oulu'], dtype=object)

In [140]:
df_municipalities[df_municipalities['MUNICIPALITY_NAME_FI'].isin(filter_unjoinable_municipalities)]

,MUNICIPALITY_NUMBER,MUNICIPALITY_NAME_FI,MUNICIPALITY_NAME_SE,MUNICIPALITY_TYPE,PRIMARY_LANGUAGE,ELECTORAL_DISTRICT_NUMBER,ELECTORAL_DISTRICT_NAME_FI,ELECTORAL_DISTRICT_NAME_SE,REGION_NUMBER,REGION_NAME_FI,REGION_NAME_SE
43,111,Heinola,Heinola,Kaupunki,suomi,6,Hämeen vaalipiiri,Tavastlands valkrets,6,Päijät-Häme,Päijänne-Tavastland
113,297,Kuopio,Kuopio,Kaupunki,suomi,9,Savo-Karjalan vaalipiiri,Savolax-Karelens valkrets,10,Pohjois-Savo,Norra Savolax
151,444,Lohja,Lojo,Kaupunki,ruotsi - suomi,2,Uudenmaan vaalipiiri,Nylands valkrets,1,Uusimaa,Nyland
183,564,Oulu,Uleåborg,Kaupunki,suomi,12,Oulun vaalipiiri,Uleåborgs valkrets,16,Pohjois-Pohjanmaa,Norra Österbotten
216,638,Porvoo,Borgå,Kaupunki,suomi,2,Uudenmaan vaalipiiri,Nylands valkrets,1,Uusimaa,Nyland


The three municipalities having `BIRTH_MUNICIPALITY_NUMBER` present in the data, but that didn't join (Heinola, Lohja, Porvoo) have differing municipality numbers. <br>
A quick check from the web (https://fi.wikipedia.org/wiki/Luettelo_Suomen_kunnista) shows that the file `municipality-listing-2020-10-21.parquet`<br>
 contains the true municipality numbers. We should fix the false values and impute missing values to make the data usable.

In [180]:
df_merge_names = df_first_names_cleaned.merge(df_municipalities, how='left', left_on='BIRTH_MUNICIPALITY_NAME', right_on='MUNICIPALITY_NAME_FI', indicator=True)

In [199]:
df_merge_names.head()

,YEAR_OF_BIRTH,BIRTH_MUNICIPALITY_NUMBER,BIRTH_MUNICIPALITY_NAME,GENDER,FIRST_NAME,AMOUNT,MUNICIPALITY_NUMBER,MUNICIPALITY_NAME_FI,MUNICIPALITY_NAME_SE,MUNICIPALITY_TYPE,PRIMARY_LANGUAGE,ELECTORAL_DISTRICT_NUMBER,ELECTORAL_DISTRICT_NAME_FI,ELECTORAL_DISTRICT_NAME_SE,REGION_NUMBER,REGION_NAME_FI,REGION_NAME_SE,_merge
0,1969.0,49.0,Espoo,1.0,Mika,39.0,49,Espoo,Esbo,Kaupunki,suomi - ruotsi,2,Uudenmaan vaalipiiri,Nylands valkrets,1,Uusimaa,Nyland,both
1,1969.0,61.0,Forssa,1.0,Marko,11.0,61,Forssa,Forssa,Kaupunki,suomi,6,Hämeen vaalipiiri,Tavastlands valkrets,4,Kanta-Häme,Egentliga Tavastland,both
2,1969.0,91.0,Helsinki,1.0,Mika,256.0,91,Helsinki,Helsingfors,Kaupunki,suomi - ruotsi,1,Helsingin vaalipiiri,Helsingfors valkrets,1,Uusimaa,Nyland,both
3,1969.0,92.0,Vantaa,1.0,Mika,47.0,92,Vantaa,Vanda,Kaupunki,suomi - ruotsi,2,Uudenmaan vaalipiiri,Nylands valkrets,1,Uusimaa,Nyland,both
4,1969.0,106.0,Hyvinkää,1.0,Mika,16.0,106,Hyvinkää,Hyvinge,Kaupunki,suomi,2,Uudenmaan vaalipiiri,Nylands valkrets,1,Uusimaa,Nyland,both


In [200]:
# Create a filter for values to be replaced
filter_replace = df_merge_names['BIRTH_MUNICIPALITY_NAME'].isin(list_unjoinable_municipalities)
filter_replace

0       False
1       False
2       False
3       False
4       False
        ...  
2340     True
2341    False
2342    False
2343    False
2344     True
Name: BIRTH_MUNICIPALITY_NAME, Length: 2345, dtype: bool

In [186]:
# Impute and replace values
df_first_names_cleaned_imputed = df_first_names_cleaned.copy()
df_first_names_cleaned_imputed.loc[filter_replace, 'BIRTH_MUNICIPALITY_NUMBER'] = df_merge_names.loc[filter_replace, 'MUNICIPALITY_NUMBER']

In [201]:
# Check that imputing & replacing values was successful
df_first_names_cleaned_imputed.loc[filter_replace, ['BIRTH_MUNICIPALITY_NUMBER', 'BIRTH_MUNICIPALITY_NAME']].drop_duplicates()

,BIRTH_MUNICIPALITY_NUMBER,BIRTH_MUNICIPALITY_NAME
20,297.0,Kuopio
25,444.0,Lohja
29,564.0,Oulu
81,638.0,Porvoo
101,111.0,Heinola
